<a href="https://colab.research.google.com/github/jrhumberto/cd/blob/main/007_NLP_spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fontes:

- https://dandaramcsousa.github.io/2020/06/11/dados-e-forro-vol-3.html
- https://colab.research.google.com/drive/1ai1x7V8xxL6geOvBdnYcWjo3p6a11Yke?usp=sharing


In [ ]:

!pip install spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv

#sp = spotipy.Spotify(client_credentials_manager= SpotifyClientCredentials(client_id="##", client_secret="###"))
sp = spotipy.Spotify(client_credentials_manager= SpotifyClientCredentials(client_id="a74e8eedfb694efc8e7162e43ec0cb3f", client_secret="bc7e77c0222741b4b2e7bebb37da69bf"))


In [ ]:
'''
 dado o id/url/urn de um artista, retorna o id 
 sp.artist(url/urn/id) retorna um objeto artista que contém as infos de um artista/banda
 nesse caso queremos apenas o id desse objeto
''' 
def get_artist_id(url):
    return sp.artist(url)['id']

In [ ]:
# dado o id do artista, retorna as faixas mais populares
def show_artist_top_tracks(id):
  response = sp.artist_top_tracks(id)
  for track in response['tracks']:
    print(track['name'])

In [ ]:
# dado o id do artista, retorna os artistas relacionados
def show_artist_related_artists(id):
  related_artists = sp.artist_related_artists(id)
  for artist in related_artists['artists']:
    print (artist['name'])

In [ ]:
# dado o id do artista, retorna todos os álbuns que o artista fez ou colaborou
def get_artist_albums(artist_id):
    albums = []
    results = sp.artist_albums(artist_id, country = 'BR')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    return albums

In [ ]:
# dada uma lista de álbuns, retorna um dicionário id: name com os álbuns autorais do artista
def get_authorial_albums(albums):
    albums_id_name = {}
    for album in albums:
        if album['album_group'] != 'appears_on':
            albums_id_name[album['id']] = album['name']
    return albums_id_name

In [ ]:
'''
 dado o id do álbum, retorna todas as músicas de um álbum de um artista 
 gera um dicionário com as seguintes informações das músicas do álbum: álbum, 
 número da música no álbum, id, nome, popularidade, duração, 
 data e artistas envolvidos
''' 
def get_album_tracks(album_id):
    spotify_album = {}
    album = sp.album(album_id)
    
    results = sp.album_tracks(album_id) # para cada álbum, retorna as informações das músicas
    tracks = results['items']
  
    for track in tracks:
        track_id = track['id']
        track_ = sp.track(track_id) # essa função tem mais info da faixa que acessando pelo álbum

        spotify_album[track_id] = {}
    
        spotify_album[track_id]['album'] = track_['album']['name']
        spotify_album[track_id]['album_type'] = album['album_type']
    
        spotify_album[track_id]['track_id'] = track_id
        spotify_album[track_id]['track_number'] = track_['track_number']
        
        spotify_album[track_id]['name'] = track_['name']
        spotify_album[track_id]['popularity'] = track_['popularity']
        spotify_album[track_id]['duration_ms'] = track_['duration_ms']
        spotify_album[track_id]['release_date'] = track_['album']['release_date']

        track_artists = track_['artists']
        
        spotify_album[track_id]['artists'] = []
        for artist in track_artists:
            spotify_album[track_id]['artists'].append(artist['name'])
        
    return spotify_album

In [ ]:
'''
 dado um dicionario com ids e nomes de álbuns (provenientes de get_authorial_albums), 
 retorna todas as músicas de todos os albuns de um artista
''' 

def get_all_albums_tracks(authorial_albums):
    spotify_albums = []
    albums_names = []
  
    for album_id, album_name in authorial_albums.items():
        print('Adicionando: ',album_id,album_name)

        if album_name not in albums_names:
            albums_names.append(album_name)
            album_tracks = get_album_tracks(album_id) 
             
        for track_infos in album_tracks.values():
            spotify_albums.append(track_infos)
            
    print('Finalizado!')
    
    return spotify_albums

In [ ]:
def convert_to_csv(filepath, name):
  keys = filepath[0].keys()
  print(keys)
  csv_name = ''+ name + '.csv'
  with open(csv_name, 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(filepath)
  return

In [ ]:
calcinhapreta = "https://open.spotify.com/artist/6Kps94g1Npexh3LrinIOvC"
artist_id = get_artist_id(calcinhapreta)
   
if artist_id:
  print("Id da Calcinha Preta: %s", artist_id) 

  print("Os artistas relacionados a Calcinha Preta segundo o Spotify:")
  show_artist_related_artists(artist_id)  
else:
  logger.error("Can't find artist: %s", artist)

In [ ]:
albums = get_artist_albums(artist_id)
authorial_albums = get_authorial_albums(albums)
print(authorial_albums)

In [ ]:
melhoralbum = get_album_tracks('5r5rYYbGdFZXincPbvekGy')
print(melhoralbum)

In [ ]:
all_albums = get_all_albums_tracks(authorial_albums)

In [ ]:
convert_to_csv(all_albums, 'calcinhapreta')

## Parte 2 - Abaixo o notebook

In [4]:
INPUT= 'https://raw.githubusercontent.com/jrhumberto/cd/main/datasets/calcinhapreta.csv'

In [5]:
import altair as alt
import pandas as pd
 
data = pd.read_csv(INPUT)

In [3]:
data.head(5)

,album,album_type,track_number,id_track,name,popularity,explicit,duration_ms,release_date,artists
0,Calcinha Preta: Edição Especial,album,1,28P5hnigrW5mQ2aZ439cEn,Quer Conhecer? Deixe!,21,False,228827,2018-08-03,['Calcinha Preta']
1,Calcinha Preta: Edição Especial,album,2,53tDhIui60U8H4CskOxOdM,Não Me Deixe Agora,24,False,293162,2018-08-03,['Calcinha Preta']
2,Calcinha Preta: Edição Especial,album,3,0OoMcqlselvg5wIhek1irs,Agora Estou Sofrendo,25,False,251580,2018-08-03,['Calcinha Preta']
3,Calcinha Preta: Edição Especial,album,4,7AARwThoxsUZ5iiIfSdkwK,Sem Explicação (I Will Be),22,False,221083,2018-08-03,['Calcinha Preta']
4,Calcinha Preta: Edição Especial,album,5,3lpaj83oEG9Sfx0vf4ckHv,Perdoar,24,False,186984,2018-08-03,['Calcinha Preta']


In [6]:
data_top20 = data.sort_values('popularity', ascending = False).head(20)
alt.Chart(data_top20).mark_circle(color = '#222437', opacity = 1).encode(
    y = alt.Y('name', axis = alt.Axis(title='Nome'), sort = None),
    x = alt.X('popularity', axis = alt.Axis(title = "Popularidade")),
    tooltip = ['name', 'popularity', 'album', 'album_type']
)

alt.Chart(...)

In [7]:
# Qual é o álbum mais popular?
data_album_most_popular = data[data['album_type'] == 'album']
data_album_most_popular = data_album_most_popular.groupby('album').median()
data_album_most_popular.sort_values(by='popularity', ascending=False)

,track_number,popularity,explicit,duration_ms
album,,,,
"Calcinha Preta, Vol. 13 (Ao Vivo)",10.0,36.0,False,230800.0
As 20 +,10.5,34.5,False,234166.5
"Ao Vivo no Recife, Vol. 18",8.0,28.0,False,215706.0
"Ao Vivo em Salvador, Vol. 1",9.0,25.0,False,236402.0
"Você Não Vale Nada, Vol. 20 (Ao Vivo)",8.5,24.0,False,201179.5
Calcinha Preta Premium,10.5,21.0,False,219546.0
Calcinha Preta: Edição Especial,6.0,21.0,False,221083.0
"Vencedor, Vol. 19",7.0,19.0,False,214453.0
"Mágica, Vol. 12",6.5,18.5,False,224809.0


In [8]:
data_groupby = pd.DataFrame(data, columns = ['album','album_type','popularity','duration_ms'])
 
data_groupby_album_chart =  data_groupby[data_groupby['album_type'] ==  'album']
 
bars = alt.Chart(data_groupby_album_chart).mark_bar(color = '#222437').encode(
    x=alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    y=alt.Y('album:O', sort='-x', axis = alt.Axis(title='Álbum'))
)
text = alt.Chart(data_groupby_album_chart).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    y=alt.Y('album:O', sort='-x', axis = alt.Axis(title='Álbum')),
    text=alt.Text('median(popularity):Q')
)
bars + text

alt.LayerChart(...)

In [9]:
data_single_most_popular = data[data['album_type'] == 'single']

bars = alt.Chart(data_single_most_popular).mark_bar(color = '#222437').encode(
    alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    alt.Y('album:O',sort = '-x',axis = alt.Axis(title='Single'))
)
text = alt.Chart(data_single_most_popular).mark_text(dx=-11, dy=3, color='white').encode(
    alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    alt.Y('album:O',sort = '-x',axis = alt.Axis(title='Single')),
    alt.Text('median(popularity):Q')
)
bars + text

alt.LayerChart(...)

In [10]:
# Qual é a maior música?

N = 10
data_shortest_songs = data.sort_values(by='duration_ms', ascending=True).head(N)
data_longest_songs = data.sort_values(by='duration_ms', ascending=False).head(N)
# ver albuns repetidos

In [11]:
chart = alt.Chart(data_shortest_songs).mark_bar(color = '#222437').encode(
    alt.Y('name:N', sort='-x', axis = alt.Axis(title='Música')),
    alt.X('duration_ms:Q', axis = alt.Axis(title='Duração(ms)')),
    tooltip = ['name', 'album']
)
text = alt.Chart(data_shortest_songs).mark_text(dx=-25, dy=3, color='white').encode(
    alt.Y('name:N', sort='-x'),
    alt.X('duration_ms:Q'),
    alt.Text('duration_ms:Q')
)

chart + text

alt.LayerChart(...)

In [12]:
# linha do tempo de lançamentos
data_time= pd.DataFrame(data, columns = ['name', 'album','album_type','release_date'])
range_ = ['#222437', '#B183AE']
alt.Chart(data_time).mark_circle(opacity = 1).encode(
    y = alt.Y('release_date:T', axis = alt.Axis(title='Lançamento')),
    x = alt.X('album:N', sort='y', axis = alt.Axis(title='Álbum')),
    color = alt.Color('album_type', legend=alt.Legend(title="Tipo do álbum"), scale=alt.Scale(range = range_)),
    tooltip = ['album','album_type','release_date:T']
)

alt.Chart(...)